In [1]:
data_var = '2023-12-16'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10212,2023-12-16,Japão B.League,03:35,Nagoya Fighting Eagles,Sendai,1.53,2.39,151.5,1.84,1.86,-4.5,1.95,1.76,46li535C,0.653595,0.418410,0.543478,0.537634,0.072005,172.214,45.895531,0.266503,2.4,1.341641,0.559017,169.51,2.270,0.372559,0.164123,29.0,172.840,24.836968,0.143699,0.6,1.341641,2.236068,166.44,2.232,0.414934,0.185902,-34.0,67,76,2.53,2.19,169.794,174.892,0.310261,0.007644,0.072426,2.62,0.524,1.011450,0.0,0.0,0.0,0.78,0.156,8.910256,0.0,0.0,0.0
10213,2023-12-16,Japão B.League,03:05,Yokohama,Diamond Dolphins,3.06,1.34,159.5,1.83,1.85,5.5,2.01,1.72,lUQ6tju4,0.326797,0.746269,0.546448,0.540541,0.073066,161.874,22.092795,0.136481,0.6,1.341641,2.236068,169.36,2.324,0.633111,0.272423,-52.0,127.980,40.586277,0.317130,1.8,1.643168,0.912871,195.03,1.584,0.344500,0.217487,-1.0,73,99,2.32,1.97,134.252,118.436,0.552829,0.007686,0.109952,-5.00,-1.000,-2.060000,0.0,0.0,0.0,0.01,0.002,170.000000,0.0,0.0,0.0
10214,2023-12-16,Japão B.League,03:05,SeaHorses Mikawa,Toyama,1.08,6.74,161.5,1.80,1.90,-15.5,2.08,1.66,dfUEvU9G,0.925926,0.148368,0.555556,0.526316,0.074294,172.150,70.997271,0.412415,2.4,1.341641,0.559017,135.29,2.018,0.742071,0.367726,50.0,427.718,190.870833,0.446254,0.6,1.341641,2.236068,703.50,5.780,3.426675,0.592850,-56.0,83,67,1.63,10.50,158.498,416.786,1.023587,0.038222,0.158815,0.60,0.120,0.666667,0.0,0.0,0.0,-1.63,-0.326,-17.607362,0.0,0.0,0.0
10215,2023-12-16,Japão B.League,03:05,NeoPhoenix,Saga,1.27,3.53,158.5,1.86,1.84,-9.5,2.03,1.70,2XUAuAfA,0.787402,0.283286,0.537634,0.543478,0.070688,141.308,25.694659,0.181834,2.4,1.341641,0.559017,168.92,1.506,0.323311,0.214682,68.0,139.430,44.856243,0.321712,1.8,1.643168,0.912871,102.06,1.816,0.652250,0.359168,33.0,103,81,1.64,1.26,121.036,242.040,0.665859,0.007644,0.125118,-0.05,-0.010,-27.000000,0.0,0.0,0.0,4.49,0.898,2.817372,0.0,0.0,0.0
10216,2023-12-16,Japão B.League,03:05,Kyoto,Utsunomiya Brex,2.84,1.39,151.5,1.88,1.79,4.5,2.05,1.68,zoTIwlPM,0.352113,0.719424,0.531915,0.558659,0.071537,219.084,106.938514,0.488116,1.2,1.643168,1.369306,231.00,2.802,1.497605,0.534477,-30.0,118.760,31.492967,0.265182,2.4,1.341641,0.559017,139.84,1.518,0.287089,0.189123,53.0,75,76,3.08,1.84,181.940,106.426,0.484778,0.034681,0.140284,1.50,0.300,6.133333,0.0,0.0,0.0,1.77,0.354,1.101695,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10540,2023-12-16,Eua Ncaa,16:00,Hofstra,Norfolk State,1.23,3.97,140.5,2.00,1.73,-10.5,1.83,1.83,0xni2XhB,0.813008,0.251889,0.500000,0.578035,0.064897,228.016,245.490520,1.076637,1.8,1.643168,0.912871,666.40,3.198,3.691757,1.154396,5.0,346.440,227.223786,0.655882,1.8,1.643168,0.912871,163.80,5.094,3.697145,0.725784,24.0,68,78,9.80,2.10,0.000,349.064,0.745182,0.102369,0.000000,0.00,0.000,inf,0.0,0.0,0.0,6.96,1.392,2.133621,0.0,0.0,0.0
10541,2023-12-16,Eua Ncaa,16:00,Hampton,James Madison,8.20,1.07,162.5,1.83,1.83,14.5,1.91,1.80,KAr8dOv5,0.121951,0.934579,0.546448,0.546448,0.056531,367.912,228.500877,0.621075,0.6,1.341641,2.236068,136.80,5.084,3.777172,0.742953,-54.0,117.030,34.073506,0.291152,3.0,0.000000,0.000000,118.44,1.384,0.289793,0.209388,70.0,7

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
10217,03:05,Japão B.League,Ibaraki Robots,Gunma,2.29,Back Home
10224,16:00,Itália Série A2,Cividale,Assigeco Piacenza,1.85,Back Home
10236,11:00,Lituânia Nkl,Kretinga,Vilniaus Stekas,1.71,Back Home
10247,02:00,Japão B2.League,Kobe,Shiga,2.34,Back Home
10254,12:00,França Lnb,Ada Blois,Cholet,1.73,Back Home
...,...,...,...,...,...,...
10521,14:30,Espanha Leb Plata,Ponferrada,Ciudad de Huelva,2.22,Back Home
10533,16:00,Eua Ncaa,Towson,Bryant University,1.73,Back Home
10534,16:00,Eua Ncaa,Presbyterian,Kennesaw State,2.18,Back Home
10536,16:00,Eua Ncaa,Michigan State,Baylor,2.18,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
10256,12:00,Finlândia I Divisão A,Oulu,Helsingin NMKY,1.87,Back Home
10459,14:30,Alemanha Bbl,Wurzburg,Bonn,1.94,Back Home
10468,18:00,Eua Ncaa,Utah State,San Francisco,1.80,Back Home
10521,14:30,Espanha Leb Plata,Ponferrada,Ciudad de Huelva,1.85,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10298,13:30,Suíça Liga Sb,Lausanne Basket,Neuchatel,2.32,Back Home
10328,08:45,Turquia Tbl,Harem Spor,Konyaspor,1.83,Back Home
10330,14:30,Turquia Superliga,Besiktas,Anadolu Efes,1.87,Back Home
10334,13:00,Noruega Blno,Asker,Bærum,2.32,Back Home
10336,12:30,Sérvia Primeira Liga,Mladost Zemun,Vojvodina Novi Sad,2.12,Back Home
10353,13:30,Polônia Liga De Basquete,Anwil Wloclawek,Ostrow Wielkopolski,1.56,Back Home
10354,23:00,Eua Ncaa,Wyoming,Weber State,1.73,Back Home
10358,10:00,Rússia Liga Vtb United,Parma Perm,Runa Basket Moscow,1.59,Back Home
10412,04:00,Coreia Do Sul Kbl,Seoul Knights,Mobis Phoebus,1.58,Back Home
10474,18:00,Eua Ncaa,Bellarmine,Evansville,1.64,Back Home
